In [2]:
# upload library
import pandas as pd
pd.set_option('display.max_columns', 50)

import numpy as np
import pickle
import matplotlib.pyplot as plt
import time

from sklearn.decomposition import NMF, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics.pairwise import cosine_similarity

In [19]:
# upload the data
df = pd.read_pickle("../../../data/prd/Paper/FR_meta_and_final_tokens_23DEC21.pkl")
df.reset_index(inplace = True, drop = True)
df.head()

,PROJECT_ID,ABSTRACT,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_START_DATE,PROJECT_END_DATE,CONTACT_PI_PROJECT_LEADER,OTHER_PIS,CONGRESSIONAL_DISTRICT,DUNS_NUMBER,ORGANIZATION_NAME,ORGANIZATION_CITY,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS,ORG_COUNT,PI_COUNT,FY_TOTAL_COST_SUM,NUM_RECORDS,final_tokens
0,89996,"This is a project to explore Game-based, Metap...",Achievement; analog; base; Cognitive Science; ...,RUI: CYGAMES: CYBER-ENABLED TEACHING AND LEARN...,NSF,NSF,NaN,0814512,9/15/2008,8/31/2012,"REESE, DEBBIE D","CARTER, BEVERLY; WOOD, CHARLES; HITT, BEN",01,068719400,WHEELING JESUIT UNIVERSITY,WHEELING,WV,26003-6243,UNITED STATES,NaN,NaN,47.076,2008,1999467.0,NaN,1,1,1999467.0,1,project explore game base metaphor enhanced ga...
1,89997,Institution: Franklin Institute Science Museum...,Active Learning; Child; Computer software; des...,ARIEL - AUGMENTED REALITY FOR INTERPRETIVE AND...,NSF,NSF,NaN,0741659,9/15/2008,8/31/2012,"SNYDER, STEVEN","ELINICH, KAREN; YOON, SUSAN",02,001741859,FRANKLIN INSTITUTE,PHILADELPHIA,PA,19103-1115,UNITED STATES,NaN,NaN,47.076,2008,1799699.0,NaN,1,1,1799699.0,1,institution franklin institute science museum ...
2,89998,Through programs (including small group conver...,Address; Age; Birth; Brain; Caregivers; Child;...,BRIGHTER FUTURES: PUBLIC DELIBERATION ABOUT TH...,NSF,NSF,NaN,0813522,9/15/2008,8/31/2011,"FINK, LAURIE KLEINBAUM","CADIGAN, KAREN; ELLENBOGEN, KIRSTEN",04,061451670,SCIENCE MUSEUM OF MINNESOTA,SAINT PAUL,MN,55102-1202,UNITED STATES,NaN,NaN,47.076,2008,1505858.0,NaN,1,1,1505858.0,1,program include small group conversation citiz...
3,89999,In partnership with the American Chemical Soci...,Advanced Development; American; Chemicals; Che...,FOSTERING US-INTERNATIONAL COLLABORATIVE PARTN...,NSF,NSF,NaN,0838627,8/1/2008,12/31/2010,"JOST, JOHN W","MILLER, BRADLEY; BOWMAN, KATHERINE",04,009059242,INTERNATIONAL UNION OF PURE AND APPLIED CHEMISTRY,DURHAM,NC,27709-3757,UNITED STATES,NaN,NaN,47.049,2008,51000.0,NaN,1,1,51000.0,1,partnership american chemical society acs nati...
4,90001,The Center for Molecular Interfacing (CMI) wil...,Address; Architecture; Carbon Nanotubes; Catal...,CCI PHASE I: CENTER FOR MOLECULAR INTERFACING,NSF,NSF,NaN,0847926,10/1/2008,9/30/2011,"ABRUNA, HECTOR D",NaN,22,872612445,CORNELL UNIVERSITY ITHACA,ITHACA,NY,14850-2820,UNITED STATES,NaN,NaN,47.049,2008,1519821.0,NaN,1,1,1519821.0,1,center molecular interfacing cmi enable integr...


In [20]:
len(df)

1143869

In [21]:
# identify abstract with the word big-data
docs = df["final_tokens"]

term_match_data = []

# big_datum: 0 matches
for i, abstract in enumerate(docs):
        if ('big datum' in abstract or 'big data' in abstract or 'big_data' in abstract):
            term_match_data.append(i)
            
# print
print('Number of abstracts with the word big-data:',len(term_match_data))

Number of abstracts with the word big-data: 4684


In [22]:
1200/len(df)

0.0010490711786052423

In [23]:
# Big Data sample
Big_data_sample = df.loc[term_match_data]
Big_data_sample.head()

,PROJECT_ID,ABSTRACT,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_START_DATE,PROJECT_END_DATE,CONTACT_PI_PROJECT_LEADER,OTHER_PIS,CONGRESSIONAL_DISTRICT,DUNS_NUMBER,ORGANIZATION_NAME,ORGANIZATION_CITY,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS,ORG_COUNT,PI_COUNT,FY_TOTAL_COST_SUM,NUM_RECORDS,final_tokens
100101,108691,Computer Science (31)This project aims to impr...,Address; Adopted; Algorithms; Area; Attention;...,A COMPREHENSIVE FRAMEWORK FOR TIMELY INTRODUCT...,NSF,NSF,NaN,0920335,9/1/2009,8/31/2012,"RAMAMURTHY, BINA","BENSCHOTEN, JOHN VAN; CHAUDHARY, VIPIN",32,038633251,STATE UNIVERSITY OF NEW YORK AT BUFFALO,BUFFALO,NY,14260-0000,UNITED STATES,NaN,NaN,47.076,2009,249875.0,NaN,1,1,249875.0,1,computer science 31this project aim improve bi...
105027,114036,The Virtualization Studio spearheads research ...,Algorithms; Arts; Behavior; Biological Preserv...,II-EN THE HUMAN VIRTUALIZATION STUDIO: FROM DI...,NSF,NSF,NaN,0855163,7/15/2009,6/30/2011,"KANADE, TAKEO","SHEIKH, YASER",14,052184116,CARNEGIE MELLON UNIVERSITY,PITTSBURGH,PA,15213-3890,UNITED STATES,NaN,NaN,47.070,2009,600000.0,NaN,1,1,600000.0,1,virtualization studio spearhead research recon...
110140,119449,Progress in the field of machine translation (...,Algorithms; Architecture; Area; cluster comput...,INCA: AN INTEGRATED CLUSTER COMPUTING ARCHITEC...,NSF,NSF,NaN,0844507,2/15/2009,1/31/2012,"VOGEL, STEPHAN","SMITH, NOAH",14,052184116,CARNEGIE MELLON UNIVERSITY,PITTSBURGH,PA,15213-3890,UNITED STATES,NaN,NaN,47.070,2009,465089.0,NaN,1,1,465089.0,1,progress field machine translation mt come dep...
153476,275113,DESCRIPTION (provided by applicant): Rare dise...,abstracting; Acquired Immunodeficiency Syndrom...,INTEGRATED DISCOVERY AND HYPOTHESIS TESTING OF...,HHS,NIH,NLM,1R01LM010140-01,7/1/2009,6/30/2011,"RABADAN, RAUL",NaN,13,621889815,COLUMBIA UNIVERSITY HEALTH SCIENCES,NEW YORK,NY,100320049,UNITED STATES,7/1/2009,6/30/2010,93.701,2009,533007.0,NaN,1,1,533007.0,1,rare disease study isolated laboratory forget ...
157841,280840,DESCRIPTION (provided by applicant): Recent ad...,Achievement; Address; Adopted; Algorithms; ant...,SEMANTIC DATA INTEGRATION FOR INTEGRATIVE CANC...,HHS,NIH,NCI,2R44CA132293-02A2,9/21/2007,8/31/2012,"KABUKA, MANSUR R.",NaN,27,035354070,INFOTECH SOFT INC,MIAMI,FL,331313207,UNITED STATES,9/21/2009,8/31/2010,93.396,2009,449882.0,NaN,1,1,449882.0,1,recent advance high throughput measurement cri...


In [24]:
# Non-big data sample
Non_big_data_sample = df.drop(term_match_data, axis=0)
non_term_match = Non_big_data_sample.index
Non_big_data_sample.head()

,PROJECT_ID,ABSTRACT,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_START_DATE,PROJECT_END_DATE,CONTACT_PI_PROJECT_LEADER,OTHER_PIS,CONGRESSIONAL_DISTRICT,DUNS_NUMBER,ORGANIZATION_NAME,ORGANIZATION_CITY,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS,ORG_COUNT,PI_COUNT,FY_TOTAL_COST_SUM,NUM_RECORDS,final_tokens
0,89996,"This is a project to explore Game-based, Metap...",Achievement; analog; base; Cognitive Science; ...,RUI: CYGAMES: CYBER-ENABLED TEACHING AND LEARN...,NSF,NSF,NaN,0814512,9/15/2008,8/31/2012,"REESE, DEBBIE D","CARTER, BEVERLY; WOOD, CHARLES; HITT, BEN",01,068719400,WHEELING JESUIT UNIVERSITY,WHEELING,WV,26003-6243,UNITED STATES,NaN,NaN,47.076,2008,1999467.0,NaN,1,1,1999467.0,1,project explore game base metaphor enhanced ga...
1,89997,Institution: Franklin Institute Science Museum...,Active Learning; Child; Computer software; des...,ARIEL - AUGMENTED REALITY FOR INTERPRETIVE AND...,NSF,NSF,NaN,0741659,9/15/2008,8/31/2012,"SNYDER, STEVEN","ELINICH, KAREN; YOON, SUSAN",02,001741859,FRANKLIN INSTITUTE,PHILADELPHIA,PA,19103-1115,UNITED STATES,NaN,NaN,47.076,2008,1799699.0,NaN,1,1,1799699.0,1,institution franklin institute science museum ...
2,89998,Through programs (including small group conver...,Address; Age; Birth; Brain; Caregivers; Child;...,BRIGHTER FUTURES: PUBLIC DELIBERATION ABOUT TH...,NSF,NSF,NaN,0813522,9/15/2008,8/31/2011,"FINK, LAURIE KLEINBAUM","CADIGAN, KAREN; ELLENBOGEN, KIRSTEN",04,061451670,SCIENCE MUSEUM OF MINNESOTA,SAINT PAUL,MN,55102-1202,UNITED STATES,NaN,NaN,47.076,2008,1505858.0,NaN,1,1,1505858.0,1,program include small group conversation citiz...
3,89999,In partnership with the American Chemical Soci...,Advanced Development; American; Chemicals; Che...,FOSTERING US-INTERNATIONAL COLLABORATIVE PARTN...,NSF,NSF,NaN,0838627,8/1/2008,12/31/2010,"JOST, JOHN W","MILLER, BRADLEY; BOWMAN, KATHERINE",04,009059242,INTERNATIONAL UNION OF PURE AND APPLIED CHEMISTRY,DURHAM,NC,27709-3757,UNITED STATES,NaN,NaN,47.049,2008,51000.0,NaN,1,1,51000.0,1,partnership american chemical society acs nati...
4,90001,The Center for Molecular Interfacing (CMI) wil...,Address; Architecture; Carbon Nanotubes; Catal...,CCI PHASE I: CENTER FOR MOLECULAR INTERFACING,NSF,NSF,NaN,0847926,10/1/2008,9/30/2011,"ABRUNA, HECTOR D",NaN,22,872612445,CORNELL UNIVERSITY ITHACA,ITHACA,NY,14850-2820,UNITED STATES,NaN,NaN,47.049,2008,1519821.0,NaN,1,1,1519821.0,1,center molecular interfacing cmi enable integr...


In [25]:
# Randomly sample N=600 abstract in each group
bg_data_training = Big_data_sample.sample(n=600,replace=False)
non_bg_data_training = Non_big_data_sample.sample(n=600,replace=False)

# combine the two dataset
Training_sample = bg_data_training
Training_sample = Training_sample.append(non_bg_data_training)

# shuffle the whole data and reset the index
Training_sample = Training_sample.sample(frac=1).reset_index(drop=True)
Training_sample.head()

,PROJECT_ID,ABSTRACT,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_START_DATE,PROJECT_END_DATE,CONTACT_PI_PROJECT_LEADER,OTHER_PIS,CONGRESSIONAL_DISTRICT,DUNS_NUMBER,ORGANIZATION_NAME,ORGANIZATION_CITY,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS,ORG_COUNT,PI_COUNT,FY_TOTAL_COST_SUM,NUM_RECORDS,final_tokens
0,715390,This project covers DCEG whole genome scan res...,15q25; 6p21; Adenocarcinoma; Age; Alcohols; Am...,GEI - GENOME-WIDE ASSOCIATION STUDIES,HHS,NIH,NCI,1ZIACP010200-06,NaN,NaN,"CAPORASO, NEIL E.",NaN,NaN,022051333,NATIONAL INSTITUTES OF HEALTH,BEHTESDA,MD,NaN,UNITED STATES,NaN,NaN,NaN,2013,159504.0,NaN,1,1,159504.0,1,project cover dceg genome scan research activi...
1,187397,DESCRIPTION (provided by applicant): Abstract ...,abstracting; Accounting; Adverse effects; Affe...,AN INTEGRATED COMPUTATIONAL AND EXPERIMENTAL P...,HHS,NIH,NIGMS,1R43GM084535-01,8/1/2008,7/31/2009,"FAMILI, IMAN",NaN,52,808128115,GT LIFE SCIENCES INC,SAN DIEGO,CA,921215782,UNITED STATES,8/1/2008,7/31/2009,93.859,2008,155737.0,NaN,1,1,155737.0,1,protein base therapeutic product contribute im...
2,801335,"Large-scale, innovative science and engineerin...",Basic Science; Big Data; Cities; cohesion; Col...,"FOSTERING SUCCESSFUL INNOVATIVE LARGE-SCALE, D...",NSF,NSF,NaN,1551609,9/1/2015,8/31/2016,"BERENTE, NICHOLAS",NaN,10,004315578,UNIVERSITY OF GEORGIA,ATHENS,GA,30602-1589,UNITED STATES,NaN,NaN,47.070,2015,100000.0,NaN,1,1,100000.0,1,large scale innovative science engineering req...
3,356403,This subproject is one of many research subpro...,absorption; Active Sites; biological systems; ...,XAS STUDIES OF THE STRUCTURE AND FUNCTION OF W...,HHS,NIH,NCRR,5P41RR001209-30 (8488),3/1/2009,2/28/2010,"KOHZUMA, TAKAMITSU",NaN,18,009214214,STANFORD UNIVERSITY,STANFORD,CA,943041222,UNITED STATES,3/1/2009,2/28/2010,93.389,2009,NaN,1662.0,1,1,0.0,1,non covalent weak interaction play important r...
4,1070923,This proposal develops novel statistics and ma...,Address; Alzheimer's Disease; Autistic Disorde...,STATISTICAL METHODS FOR ULTRAHIGH-DIMENSIONAL ...,HHS,NIH,NIGMS,2R01GM072611-13,2/1/2006,1/31/2022,"FAN, JIANQING",NaN,12,002484665,PRINCETON UNIVERSITY,PRINCETON,NJ,085430036,UNITED STATES,2/1/2018,1/31/2019,93.859,2018,308503.0,NaN,1,1,308503.0,1,proposal develop novel statistic machine learn...


In [8]:
# count the number of abstracts in the training sample
print('Number of abstracts in the training sample:',len(Training_sample))

Number of abstracts in the training sample: 1200


In [9]:
# split and save the file into csv
sampling = Training_sample[['PROJECT_ID', 'ABSTRACT']]
sampling['Is it related to Big-Data'] = ""
sampling['Expert name'] = ""

#for i in range(1,13):
#    trunc_sample = sampling.sample(n=100)
#    trunc_sample.to_csv('training_distinct_abstract_'+str(i)+'.csv')
#    sampling = sampling.drop(trunc_sample.index)
    
    

<ipython-input-9-93bf926626e3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampling['Is it related to Big-Data'] = ""
<ipython-input-9-93bf926626e3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampling['Expert name'] = ""


In [10]:
# create training set with an Identifier which states if the abstacts is duplicated across group.
sampling = Training_sample[['PROJECT_ID', 'ABSTRACT']]
sampling['Duplicated'] = 0
sampling['Is it related to Big-Data'] = ""
sampling['Expert name'] = ""

sampling0 = sampling

#for i in range(1,13):
#    trunc_sample = sampling.sample(n=100)
#    trunc_sample.to_csv('training_distinct_abstract_'+str(i)+'.csv')
#    sampling = sampling.drop(trunc_sample.index)

<ipython-input-10-dc3cea04e9f7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampling['Duplicated'] = 0
<ipython-input-10-dc3cea04e9f7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampling['Is it related to Big-Data'] = ""
<ipython-input-10-dc3cea04e9f7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

# After labelling by Expert

In [8]:
# Combine the labelled training set
# create an empty dataframe and a list of id for the label group
#group = list(range(1,13))
group = [1, 2, 3, 4, 5, 6, 7, 8, 10, 9, 11, 12,]
df = pd.DataFrame()

for gid in group:
    # Load each file
    group_df = pd.read_excel('/project/biocomplexity/sdad/projects_data/ncses/prd/Digital_abstract_labelled/training_distinct_abstract_'+str(gid)+'.xlsx')
    df = df.append(group_df)

df.head()    

,Duplicates,index,PROJECT_ID,ABSTRACT,Is it related to Big-Data,Expert name
0,0,979,950056,ï»¿ DESCRIPTION (provided by applicant): Adv...,1,Kristian
1,0,816,1178172,Project Summary/AbstractThe establishment of t...,1,Kristian
2,0,1096,163037,DESCRIPTION (provided by applicant): The long-...,0,Kristian
3,0,725,880197,DESCRIPTION (provided by applicant): The most ...,0,Kristian
4,0,92,887331,Transsphenoidal surgery (TSS) is the best trea...,0,Kristian


In [9]:
# Remove the duplicates and non filing abstracts
subset = df[(df['Duplicates']==0)]
print('Number of non-duplicated', len(subset))

subset = subset[~subset['Is it related to Big-Data'].isnull()]
print('Number of labelled abstracts', len(subset))

Number of non-duplicated 1200
Number of labelled abstracts 1200


In [10]:
# Add label
subset['label'] = 'Non Big-data'
subset.loc[subset['Is it related to Big-Data']==1,'label'] = 'Big-data'

In [11]:
subset.to_csv('/project/biocomplexity/sdad/projects_data/ncses/prd/Digital_abstract_labelled/labelled_abstracts.csv')

In [12]:
# save the data in the pickle file
import pickle

subset.to_pickle('/project/biocomplexity/sdad/projects_data/ncses/prd/Digital_abstract_labelled/labelled_abstracts.pkl')

In [13]:
subset

,Duplicates,index,PROJECT_ID,ABSTRACT,Is it related to Big-Data,Expert name,label
0,0,979,950056,ï»¿ DESCRIPTION (provided by applicant): Adv...,1,Kristian,Big-data
1,0,816,1178172,Project Summary/AbstractThe establishment of t...,1,Kristian,Big-data
2,0,1096,163037,DESCRIPTION (provided by applicant): The long-...,0,Kristian,Non Big-data
3,0,725,880197,DESCRIPTION (provided by applicant): The most ...,0,Kristian,Non Big-data
4,0,92,887331,Transsphenoidal surgery (TSS) is the best trea...,0,Kristian,Non Big-data
...,...,...,...,...,...,...,...
95,0,83,1226292,The candidate for this Mentored Career Develop...,1,Kathryn,Big-data
96,0,519,937305,ï»¿ DESCRIPTION (provided by applicant): The...,1,Kathryn,Big-data
97,0,948,278787,DESCRIPTION (provided by applicant): All cance...,0,Kathryn,Non Big-data
98,0,1104,1098499,The NSF Big Data Innovation Hubs (Big Data Hub...,1,Kathryn,Big-data


In [14]:
# sample comparison
dup_df = df[(df['Duplicates']==1)]
dup_df = dup_df.rename(columns = {'Is it related to Big-Data': 'Decision_2', 'Expert name': 'Expert name 2'}, inplace = False)
dup_df = dup_df[['PROJECT_ID', 'Decision_2', 'Expert name 2']]
dup_df

,PROJECT_ID,Decision_2,Expert name 2
100,681368,0,Kristian
101,800992,0,Kristian
102,1095672,0,Kristian
103,980662,0,Kristian
104,1061272,0,Kristian
...,...,...,...
105,1143102,0,Kathryn
106,1244969,0,Kathryn
107,795854,1,Kathryn
108,866215,1,Kathryn


In [15]:
# subset df to the non duplicates value with the same id value  in Project ID
Proj_ID = list(dup_df['PROJECT_ID'])
dup_subset = subset[subset['PROJECT_ID'].isin(Proj_ID)]
dup_subset = dup_subset.rename(columns = {'Is it related to Big-Data': 'Decision_1', 'Expert name': 'Expert name 1'}, inplace = False)
dup_subset = dup_subset[['PROJECT_ID', 'ABSTRACT', 'Decision_1', 'Expert name 1']]
dup_subset

,PROJECT_ID,ABSTRACT,Decision_1,Expert name 1
0,950056,ï»¿ DESCRIPTION (provided by applicant): Adv...,1,Kristian
1,1178172,Project Summary/AbstractThe establishment of t...,1,Kristian
2,163037,DESCRIPTION (provided by applicant): The long-...,0,Kristian
3,880197,DESCRIPTION (provided by applicant): The most ...,0,Kristian
4,887331,Transsphenoidal surgery (TSS) is the best trea...,0,Kristian
...,...,...,...,...
5,966523,The University of Florida (UF) requests funds ...,1,Kathryn
6,477602,DESCRIPTION (provided by applicant): A key unr...,0,Kathryn
7,351910,DESCRIPTION (provided by applicant): Raci...,1,Kathryn
8,527734,DESCRIPTION (provided by applicant): The integ...,0,Kathryn


In [16]:
# merge the two data
Test_data = dup_subset.merge(dup_df, how='left', on='PROJECT_ID')
Test_data

,PROJECT_ID,ABSTRACT,Decision_1,Expert name 1,Decision_2,Expert name 2
0,950056,ï»¿ DESCRIPTION (provided by applicant): Adv...,1,Kristian,1,Alan
1,1178172,Project Summary/AbstractThe establishment of t...,1,Kristian,1,Alan
2,163037,DESCRIPTION (provided by applicant): The long-...,0,Kristian,0,Alan
3,880197,DESCRIPTION (provided by applicant): The most ...,0,Kristian,0,Alan
4,887331,Transsphenoidal surgery (TSS) is the best trea...,0,Kristian,0,Alan
...,...,...,...,...,...,...
116,966523,The University of Florida (UF) requests funds ...,1,Kathryn,1,NaN
117,477602,DESCRIPTION (provided by applicant): A key unr...,0,Kathryn,0,NaN
118,351910,DESCRIPTION (provided by applicant): Raci...,1,Kathryn,1,NaN
119,527734,DESCRIPTION (provided by applicant): The integ...,0,Kathryn,0,NaN


In [17]:
pd.crosstab(Test_data.Decision_1, Test_data.Decision_2, margins=False, margins_name="Total")

Decision_2,0,1
Decision_1,,
0,64,14
1,11,32


In [26]:
# Compare the labelling with the term match
Big_data_sample['PROJECT_ID'] = pd.to_numeric(Big_data_sample['PROJECT_ID'])
term_match_id = list(Big_data_sample['PROJECT_ID'])

subset['Term matching'] = 0
subset.loc[subset['PROJECT_ID'].isin(term_match_id),'Term matching'] = 1

In [27]:
# Cross table term-matching and labelling
pd.crosstab(subset['Is it related to Big-Data'], subset['Term matching'], margins=False, normalize='index', margins_name="Total")

Term matching,0,1
Is it related to Big-Data,,
0,0.780591,0.219409
1,0.088296,0.911704
o,1.000000,0.000000


In [28]:
pd.crosstab(subset['Is it related to Big-Data'], subset['Term matching'], margins=False, margins_name="Total")

Term matching,0,1
Is it related to Big-Data,,
0,555,156
1,43,444
o,2,0
